In [0]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{||\mathbf{x} - \mathbf{y}||}
$$

In [0]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
        
    if len(vec)==1:
      return vec,np.eye(1)
    
    norm=np.linalg.norm(vec)
    outvec=np.array([0]*len(vec))*1.0
    outvec[0]=norm
    u=(vec-outvec)/np.linalg.norm(vec-outvec)
    uu_T=np.outer(u,u)
    E=np.eye(uu_T.shape[0])*1.0
    H=E-2*uu_T
    
    return outvec,H
    
    # ENTER YOUR CODE HERE

In [174]:
# Test (marked)

v = np.array([1, 2, 3])*1.0
v1, h = householder(v)
assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1)

AssertionError: ignored

Assert не проходит, но видно, что на самом деле векторы совпадают.

In [0]:
rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the QR decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $QR$ decomposition of $\mathbf{A}$. 

In [0]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
  
    # ENTER YOUR CODE HERE
    R=a1
    Q=np.eye(m,m)
    for i in range(min(m,n)):
      a_i=R[i:,i:]
      H_i=householder(a_i[:,0])[1]
      H_i_big=np.eye(m,m)
      H_i_big[i:,i:]=H_i
      Q=np.dot(H_i_big,Q)
      R=np.dot(H_i_big,R)
    
    return np.linalg.inv(Q),R

In [0]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [0]:
# Test (marked)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [0]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

In [180]:
print(a)

[[0.19151945 0.62210877 0.43772774]
 [0.78535858 0.77997581 0.27259261]
 [0.27646426 0.80187218 0.95813935]
 [0.87593263 0.35781727 0.50099513]
 [0.68346294 0.71270203 0.37025075]]


In [181]:
print(r)
print(rr)

[[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [-0.         -0.          0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.         -0.         -0.        ]]
[[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]


In [182]:
print(np.abs(r)-np.abs(rr))

[[ 0. -0. -0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


In [183]:
print(q)
print(qq)

[[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  0.52144688  0.28453698  0.04822969]
 [ 0.48765568  0.12171264 -0.27224305 -0.47049398  0.67223293]]
[[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]]


In [184]:
print(np.abs(q)-np.abs(qq))

[[-0.         -0.         -0.          0.1080946  -0.19289592]
 [ 0.          0.          0.         -0.50637718  0.28376255]
 [ 0.          0.         -0.         -0.05494342  0.0980471 ]
 [ 0.          0.         -0.          0.09486041 -0.16927938]
 [ 0.          0.          0.          0.14275173 -0.0799949 ]]


Check if your `q` and `r` agree with `qq` and `rr`. Explain.

*Enter your explanation here (Russian or English, up to you)*

Как видно, с точностью до знаков некоторых элементов наша матрица $r$ и библиотечная $rr$ совпвдвют.

Аналогично совпадают левые части матриц $q$ и $qq$ (те, которые участвуют в thin qr разложении). Оставшиеся две колонки, которые не совпали,  при разложении умножаются на нули, поэтому важной роли не играют.

Также, при произведении $qq$ и $rr$ знаки, которые поменялись, компенсируют друг друга, и в результате тоже получается матрица $a$.

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. Write a second function, which uses the reflection vertices to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests.

In [0]:
def get_u(vec):
     """Construct normal vector u.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    u : array of floats, shape (n,)
        Vector, with is used to transform vec to outvec: ``outvec[1:]==0`` and ``|outvec| == |vec|``
    
    """
  
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
        
    if len(vec)==1:
      return vec
    
    norm=np.linalg.norm(vec)
    outvec=np.array([0]*len(vec))*1.0
    outvec[0]=norm
    u=(vec-outvec)/np.linalg.norm(vec-outvec)

    return u

In [0]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix without Householder matrixes.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    U : list of ndarrays of shape(m)
        The list of used vectors u
    R : ndarray, shape(m, n)
        The upper triangular matrix
    
    """
    
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
  
    # ENTER YOUR CODE HERE
    R=a1*1.0
    U=[]
    for i in range(min(m,n)):
      a_i=R[i:,i:]
      u=get_u(a_i[:,0])
      u_big=np.zeros(m)
      u_big[i:]=u
      U.append(u_big)
      R-=2*np.outer(u_big,np.dot(u_big,R))
    
    return U,R

In [0]:
def mult_by_Q_T(a1,U):
  """Последовательно преобразуем матрицу а1 к матрице R, используя найденные векторы u.
  Применение этой функции к матрице равносильно умножению слева на матрицу Q_T"""
  
  a=a1*1.0
  for u in U:
    a-=2*np.outer(u,np.dot(u,a))
  return a

In [188]:
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
U, r_new = qr_decomp(a)

print(r-r_new)

[[ 0. -0. -0.]
 [ 0.  0.  0.]
 [-0.  0.  0.]
 [ 0. -0.  0.]
 [-0.  0. -0.]]


Как видно, полученная матрица R совпадает с матрицей из предыдущего задания.

Проверим, что функция произведений тоже даёт верный результат.

In [189]:
r_new_new=mult_by_Q_T(a,U)
print(r-r_new_new)

[[ 0. -0. -0.]
 [ 0.  0.  0.]
 [-0.  0.  0.]
 [ 0. -0.  0.]
 [-0.  0. -0.]]


Снова всё верно.